In [2]:
import subprocess
import glob
import os
from os.path import exists
import re
import pandas as pd
from multiprocessing import Pool
# import pyreadr

#### the genome, individual bams and so on have been prepared in the previous diffTF, mainly addiing the prefix "chr"


### add "chr" prefix to chromosome names in GTF

In [5]:
! awk '!/^#/ {$1 = "chr"$1} {print}' /media/HDD2/Genomes/Ath_Ensembl56/Arabidopsis_thaliana.TAIR10.56.gtf > ./TOBIAS/refs_modified/Arabidopsis_thaliana.TAIR10.56_modified.gtf

In [14]:
! grep -v '^#' /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/refs_modified/Arabidopsis_thaliana.TAIR10.56_modified.gtf > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/refs_modified/Arabidopsis_thaliana.TAIR10.56_no_header.gtf

In [17]:
! awk 'BEGIN{OFS="\t"} {$1=$1; print}' /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/refs_modified/Arabidopsis_thaliana.TAIR10.56_no_header.gtf > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/refs_modified/Arabidopsis_thaliana.TAIR10.56_no_header_tabbed.gtf


#### merge bams

In [2]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/C12*.bam

In [3]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C14_merged.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/C1*.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/C2*.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/C3*.bam

In [4]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/DM*.bam

In [5]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM6_DM7_merged.bam /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/data/4_intersect_bam_modified/OK*.bam

In [6]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_C14_merged.bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C14_merged.bam 

In [7]:
! samtools merge -@ 80 /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_DM6_DM7.bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM6_DM7_merged.bam

In [11]:
def index_bam(bam_file):
    try:
        print(f"Indexing {bam_file}...")
        subprocess.run(["samtools", "index",
                        '-@', '30',
                        bam_file], check=True)
        print(f"Finished indexing {bam_file}.")
    except subprocess.CalledProcessError as e:
        print(f"Error indexing {bam_file}: {e}")

# Path to the directory containing your BAM files
directory = '/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam'

# List of all BAM files in the directory
bam_files = [os.path.join(directory, f) for f in os.listdir(directory) if f.endswith('.bam')]

# Number of worker processes to use
num_workers = 3

# Create a pool of worker processes
with Pool(num_workers) as pool:
    # Map the index_bam function to each BAM file
    pool.map(index_bam, bam_files)

Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam...Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam...Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_DM6_DM7.bam...


Finished indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_merged.bam.
Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM6_DM7_merged.bam...
Finished indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_merged.bam.
Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_C14_merged.bam...
Finished indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM6_DM7_merged.bam.
Indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C14_merged.bam...
Finished indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_DM6_DM7.bam.
Finished indexing /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_C14_merged.bam.
Fin

#### TOBIAS use broad peaks, refer to 1_DM1_DM2d_ATAC_call_peak.ipynb and https://github.com/loosolab/TOBIAS/wiki/test-data

In [49]:
! cat /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad/C12A_clean_filtered_peaks.broadPeak /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad/C12B_clean_filtered_peaks.broadPeak /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad/C12C_clean_filtered_peaks.broadPeak | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C12_broad_peaks.bed

In [50]:
! cat /media/HDD3/bulk_ATAC_DMs/DM1_DM2d_Novogene/X401SC23101639-Z01-F001/Analysi_dir/12_macs2peaks_broad/DM*.broadPeak | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/DM1_DM2d_broad_peaks.bed

In [51]:
! cat /media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/12_macs2peaks_broad/OK*.broadPeak | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/DM6_DM7_broad_peaks.bed

In [52]:
! cat /media/HDD3/bulk_ATAC_DMs/N2315964_30-927980316_Lib_2023-09-30/Analysi_dir/12_macs2peaks_broad/C*.broadPeak | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C14_broad_peaks.bed

#### further merge bed peaks files by DM or Col

In [53]:
cat /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C*.bed | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C12_C14_merged_peaks.bed

In [54]:
cat /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/D*.bed | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/DM1_DM2d_DM6_DM7_merged_peaks.bed

In [55]:
cat /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/DM1_DM2d_DM6_DM7_merged_peaks.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/C12_C14_merged_peaks.bed | bedtools sort | bedtools merge > /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/all_merged_peaks.bed

##### add "chr" to the first chromesome names

In [56]:
! awk '{print "chr"$0}' TOBIAS/merged_peaks/all_merged_peaks.bed > TOBIAS/merged_peaks/chr_all_merged_peaks.bed

### Annotation of peaks to nearest genes

#### download gtf from https://ftp.ensemblgenomes.ebi.ac.uk/pub/plants/release-58/gtf/arabidopsis_thaliana/Arabidopsis_thaliana.TAIR10.58.gtf.gz, then grep -v "^#" Arabidopsis_thaliana.TAIR10.58.gtf | awk '{print "chr"$0}' > Arabidopsis_thaliana.TAIR10.58_modified.gtf, use it for this

In [57]:
! uropa --bed TOBIAS/merged_peaks/chr_all_merged_peaks.bed --gtf /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58.gtf --show_attributes gene_id gene_name --feature_anchor start --distance 20000 10000 --feature gene -o ./TOBIAS/UROPA/

2024-01-24 17:26:38 (283004) [INFO]	Started UROPA 4.0.3
2024-01-24 17:26:38 (283004) [INFO]	Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
2024-01-24 17:26:38 (283004) [INFO]	Command-line call: /home/hu/miniconda3/envs/work2/bin/uropa --bed TOBIAS/merged_peaks/chr_all_merged_peaks.bed --gtf /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58.gtf --show_attributes gene_id gene_name --feature_anchor start --distance 20000 10000 --feature gene -o ./TOBIAS/UROPA/
2024-01-24 17:26:38 (283004) [INFO]	Reading configuration from commandline/input config
2024-01-24 17:26:38 (283004) [INFO]	Preparing .gtf-file for fast access
[E::hts_idx_push] Unsorted positions on sequence #1: 29677904 followed by 26964087
2024-01-24 17:26:41 (283004) [WARNING]	Indexing failed - the GTF is probably unsorted
2024-01-24 17:26:41 (283004) [WARNING]	Attempting to sort with call: grep -v "^#" ./TOBIAS/UROPA/chr_all_merged_peaks_feature_subset.gtf | sort -k1,1 -k4,4n > ./TOBIA

#### the file is chr_all_merged_peaks_finalhits.txt

In [62]:
! cut -f 1-6,16-17 TOBIAS/UROPA/chr_all_merged_peaks_finalhits.txt | head -n 1 > TOBIAS/UROPA/merged_peaks_annotated_header.txt

cut: write error: Broken pipe


In [61]:
! cut -f 1-6,16-17 TOBIAS/UROPA/chr_all_merged_peaks_finalhits.txt | tail -n +2 > TOBIAS/UROPA/merged_peaks_annotated.bed

#### ATACorrect

In [12]:
! TOBIAS ATACorrect --bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_C14_merged.bam --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed --outdir ./TOBIAS/ATACorrect_C12_C14 --cores 50

# TOBIAS 0.16.1 ATACorrect (run started 2024-01-24 16:32:25.136960)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS ATACorrect --bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_C14_merged.bam --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed --outdir ATACorrect_C12_C14 --cores 50

# ----- Input parameters -----
# bam:	/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/C12_C14_merged.bam
# genome:	/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa
# peaks:	/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed
# regions_in:	None
# regions_out:	None
# blacklist:	None
# extend:	100
# split_strands:	False
# norm_off:	False
# track_off:	[]
# drop_chroms:	['chrM

In [4]:
! TOBIAS ATACorrect --bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM1_DM2d_DM6_DM7.bam --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed --outdir ./TOBIAS/ATACorrect_DM --cores 50

# TOBIAS 0.16.1 ATACorrect (run started 2024-01-24 15:34:26.940923)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS ATACorrect --bam /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM_merged.bam --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed --outdir ATACorrect_test --cores 50

# ----- Input parameters -----
# bam:	/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_bam/DM_merged.bam
# genome:	/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa
# peaks:	/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed
# regions_in:	None
# regions_out:	None
# blacklist:	None
# extend:	100
# split_strands:	False
# norm_off:	False
# track_off:	[]
# drop_chroms:	['chrM', 'chrMT', '

#### ScoreBigwig

In [18]:
! TOBIAS FootprintScores --signal TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --regions TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed --output TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --cores 50

# TOBIAS 0.16.1 ScoreBigwig (run started 2024-01-24 16:57:03.896516)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS FootprintScores --signal TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --regions TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed --output TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --cores 50

# ----- Input parameters -----
# signal:	TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw
# output:	TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw
# regions:	TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed
# score:	footprint
# absolute:	False
# extend:	100
# smooth:	1
# min_limit:	None
# max_limit:	None
# fp_min:	20
# fp_max:	50
# flank_min:	10
# flank_max:	30
# window:	100
# cores:	50
# split:	100
# verbosity:	3


# ----- Output files -----
# TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw


2024-01-24 16:57:03 (278388) [INFO]	Processing input files
2024-01-24 16:57:03 (278388) [INFO]	- Opening input cutsite bigwig
2024-

In [19]:
! TOBIAS FootprintScores --signal TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --regions TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed --output TOBIAS/Footprints/C12_C14_footprint.bw --cores 50

# TOBIAS 0.16.1 ScoreBigwig (run started 2024-01-24 16:58:33.845899)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS FootprintScores --signal TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --regions TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed --output TOBIAS/Footprints/C12_C14_footprint.bw --cores 50

# ----- Input parameters -----
# signal:	TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw
# output:	TOBIAS/Footprints/C12_C14_footprint.bw
# regions:	TOBIAS/merged_peaks/chr_DM_and_Col_merged_peaks.bed
# score:	footprint
# absolute:	False
# extend:	100
# smooth:	1
# min_limit:	None
# max_limit:	None
# fp_min:	20
# fp_max:	50
# flank_min:	10
# flank_max:	30
# window:	100
# cores:	50
# split:	100
# verbosity:	3


# ----- Output files -----
# TOBIAS/Footprints/C12_C14_footprint.bw


2024-01-24 16:58:33 (279541) [INFO]	Processing input files
2024-01-24 16:58:33 (279541) [INFO]	- Opening input cutsite bigwig
2024-01-24 16:58:33 (27954

In [3]:
motifs = '/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme'
fasta = '/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa'

In [5]:
import subprocess

# Define the command as a list of arguments
cmd = [
    "TOBIAS", "BINDetect", 
    "--motifs", motifs,
    "--signals", 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw', 'TOBIAS/Footprints/C12_C14_footprint.bw',
    "--genome", fasta,
    "--peaks", "TOBIAS/UROPA/merged_peaks_annotated.bed",
    "--peak_header", "TOBIAS/UROPA/merged_peaks_annotated_header.txt",
    "--outdir", "TOBIAS/BINDetect_output",
    "--cond_names", "DM", "COL",
    "--cores", "99"
]

# Execute the command
try:
    subprocess.run(cmd, check=True)
    info = 'tobias BINDetect Success'
except subprocess.CalledProcessError as e:
    print(f"An error occurred: {e}")
    info = 'tobias BINDetect Failed'

# TOBIAS 0.16.1 BINDetect (run started 2024-01-24 18:38:09.367005)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS BINDetect --motifs /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme --signals TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw TOBIAS/Footprints/C12_C14_footprint.bw --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks TOBIAS/UROPA/merged_peaks_annotated.bed --peak_header TOBIAS/UROPA/merged_peaks_annotated_header.txt --outdir TOBIAS/BINDetect_output --cond_names DM COL --cores 99

# ----- Input parameters -----
# signals:	['TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw', 'TOBIAS/Footprints/C12_C14_footprint.bw']
# peaks:	TOBIAS/UROPA/merged_peaks_annotated.bed
# motifs:	['/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme']
# genome:	/media/HDD2/donghui/diffTF_ath/diffTF/example/st

/home/hu/miniconda3/envs/TOBIAS_ENV/lib/python3.10/site-packages/logomaker/src/matrix.py:270: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.34494196 0.08623549 0.00165837 0.51409619 0.00165837 0.99502488
 0.00331675 0.0066335  0.00165837 0.10281924 0.19402985 0.1641791
 0.31177446 0.41127695]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prob_df.loc[:, :] = vals / vals.sum(axis=1)[:, np.newaxis]
/home/hu/miniconda3/envs/TOBIAS_ENV/lib/python3.10/site-packages/logomaker/src/matrix.py:270: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.00166113 0.00166113 0.00166113 0.23920266 0.02491694 0.13289037
 0.08139535 0.26079734 0.54152824 0.00166113 0.99501661 0.99501661
 0.00166113 0.06478405]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prob_df.loc[:, :] = va

2024-01-24 18:38:11 (306330) [INFO]	Progress 0.16%
2024-01-24 18:38:11 (306330) [INFO]	Progress 0.81%
2024-01-24 18:38:11 (306330) [INFO]	Progress 2.42%


/home/hu/miniconda3/envs/TOBIAS_ENV/lib/python3.10/site-packages/logomaker/src/matrix.py:270: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.49913345 0.57885615 0.50259965 0.82668977 0.78336222 0.98786828
 0.83362218 0.99133449 0.90814558 0.95320624 0.85095321 0.91334489
 0.67244367 0.68457539 0.66551127]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prob_df.loc[:, :] = vals / vals.sum(axis=1)[:, np.newaxis]
/home/hu/miniconda3/envs/TOBIAS_ENV/lib/python3.10/site-packages/logomaker/src/matrix.py:270: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.15771231 0.24263432 0.3847487  0.05545927 0.04679376 0.00693241
 0.10745234 0.0034662  0.0779896  0.0017331  0.02946274 0.01213172
 0.05719237 0.20970537 0.09012132]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
 

2024-01-24 18:38:11 (306330) [INFO]	Progress 6.62%
2024-01-24 18:38:11 (306330) [INFO]	Progress 12.44%


...

2024-01-24 18:38:13 (306330) [INFO]	Progress 83.84%
2024-01-24 18:38:14 (306330) [INFO]	Progress 88.21%


/home/hu/miniconda3/envs/TOBIAS_ENV/lib/python3.10/site-packages/logomaker/src/matrix.py:270: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.02209945 0.21178637 0.33701657 0.4198895  0.16022099 0.17311234
 0.0626151  0.00184162 0.24677716 0.95395948 0.06077348]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prob_df.loc[:, :] = vals / vals.sum(axis=1)[:, np.newaxis]
/home/hu/miniconda3/envs/TOBIAS_ENV/lib/python3.10/site-packages/logomaker/src/matrix.py:270: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.95027624 0.03314917 0.19152855 0.08655617 0.07550645 0.46040516
 0.56906077 0.99447514 0.03499079 0.00184162 0.01289134]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  prob_df.loc[:, :] = vals / vals.sum(axis=1)[:, np.newaxis]
/home/hu/miniconda3/envs/TOBIA

2024-01-24 18:38:14 (306330) [INFO]	Progress 91.76%
2024-01-24 18:38:14 (306330) [INFO]	Progress 94.35%
2024-01-24 18:38:14 (306330) [INFO]	Progress 97.58%
2024-01-24 18:38:14 (306330) [INFO]	Progress 98.22%
2024-01-24 18:38:14 (306330) [INFO]	Progress 99.68%
2024-01-24 18:38:14 (306330) [INFO]	Progress 99.84%
2024-01-24 18:38:14 (306330) [INFO]	Progress 100.0%
2024-01-24 18:38:14 (306330) [INFO]	Progress done!


2024-01-24 18:38:14 (306330) [INFO]	Scanning for motifs and matching to signals...
2024-01-24 18:38:14 (306330) [INFO]	Progress 0%
2024-01-24 18:38:22 (306330) [INFO]	Progress 2.0%
2024-01-24 18:38:22 (306330) [INFO]	Progress 3.0%
2024-01-24 18:38:22 (306330) [INFO]	Progress 4.0%
2024-01-24 18:38:23 (306330) [INFO]	Progress 5.0%
2024-01-24 18:38:23 (306330) [INFO]	Progress 6.0%
2024-01-24 18:38:23 (306330) [INFO]	Progress 9.0%
2024-01-24 18:38:24 (306330) [INFO]	Progress 11.0%
2024-01-24 18:38:24 (306330) [INFO]	Progress 12.0%
2024-01-24 18:38:25 (306330) [INFO]	Progress 13.0%

/home/hu/miniconda3/envs/TOBIAS_ENV/lib/python3.10/site-packages/tobias/tools/bindetect.py:759: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  info_table.at[names[i], base + "_highlighted"] = False
/home/hu/miniconda3/envs/TOBIAS_ENV/lib/python3.10/site-packages/tobias/tools/bindetect.py:759: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  info_table.at[names[i], base + "_highlighted"] = False
/home/hu/miniconda3/envs/TOBIAS_ENV/lib/python3.10/site-packages/tobias/tools/bindetect.py:757: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated a

An error occurred: Command '['TOBIAS', 'BINDetect', '--motifs', '/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/scan_motif/Ath_TF_binding_motifs_plantTFDB.meme', '--signals', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw', 'TOBIAS/Footprints/C12_C14_footprint.bw', '--genome', '/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa', '--peaks', 'TOBIAS/UROPA/merged_peaks_annotated.bed', '--peak_header', 'TOBIAS/UROPA/merged_peaks_annotated_header.txt', '--outdir', 'TOBIAS/BINDetect_output', '--cond_names', 'DM', 'COL', '--cores', '99']' returned non-zero exit status 1.


#### BINDetect this command need to be run in a new env, use lower version of matplotlib

In [6]:
TOBIAS BINDetect --motifs ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme --signals TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks TOBIAS/UROPA/merged_peaks_annotated.bed --peak_header TOBIAS/UROPA/merged_peaks_annotated_header.txt --outdir TOBIAS/BINDetect_output_v3 --cond_names COL DM --cores 99

3.8.2


#### Visualize the difference in footprints between two conditions for all accessible sites:

In [18]:
! TOBIAS PlotAggregate --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_all.bed --signals TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --output TOBIAS/PlotAggregate/DOF4.7_AT4G38000_footprint_comparison_all.pdf --share_y both --plot_boundaries --signal-on-x

# TOBIAS 0.16.1 PlotAggregate (run started 2024-01-30 14:19:10.680743)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotAggregate --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_all.bed --signals TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --output TOBIAS/PlotAggregate/DOF4.7_AT4G38000_footprint_comparison_all.pdf --share_y both --plot_boundaries --signal-on-x

# ----- Input parameters -----
# TFBS:	['TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_all.bed']
# signals:	['TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw', 'TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw']
# regions:	[]
# whitelist:	[]
# blacklist:	[]
# output:	TOBIAS/PlotAggregate/DOF4.7_AT4G38000_footprint_comparison_all.pdf
# output_txt:	None
# title:	Aggregated signals
# flank:	60
# TFBS_labels:	None
# signal_labels:	None
# region_labels:	None
# share_y:	both
# nor

In [22]:
import subprocess

def compare_all_between_two_conditions(tf_name):
    # Define the base path for BED files and signal files
    bed_file = f"TOBIAS/BINDetect_output_v3/{tf_name}/beds/{tf_name}_all.bed"
    signal_dm = "TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw"
    signal_col = "TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw"

    # Define the command for PlotAggregate
    cmd = [
        "TOBIAS", "PlotAggregate", 
        "--TFBS", bed_file,
        "--signals", signal_dm, signal_col,
        "--output", f"TOBIAS/PlotAggregate/{tf_name}_footprint_comparison_all.pdf",
        "--share_y", "both", 
        "--plot_boundaries", 
        "--signal-on-x"
    ]

    # Execute the command
    try:
        subprocess.run(cmd, check=True)
        return f'TOBIAS PlotAggregate for {tf_name} comparing all sites between conditions Success'
    except subprocess.CalledProcessError as e:
        return f"An error occurred: {e}"
compare_all_between_two_conditions("DOF4.7_AT4G38000")

# TOBIAS 0.16.1 PlotAggregate (run started 2024-01-30 14:43:09.833908)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotAggregate --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_all.bed --signals TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --output TOBIAS/PlotAggregate/DOF4.7_AT4G38000_footprint_comparison_all.pdf --share_y both --plot_boundaries --signal-on-x

# ----- Input parameters -----
# TFBS:	['TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_all.bed']
# signals:	['TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw', 'TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw']
# regions:	[]
# whitelist:	[]
# blacklist:	[]
# output:	TOBIAS/PlotAggregate/DOF4.7_AT4G38000_footprint_comparison_all.pdf
# output_txt:	None
# title:	Aggregated signals
# flank:	60
# TFBS_labels:	None
# signal_labels:	None
# region_labels:	None
# share_y:	both
# nor

'TOBIAS PlotAggregate for DOF4.7_AT4G38000 comparing all sites between conditions Success'

In [21]:
! TOBIAS PlotAggregate --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bound.bed  TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed --signals TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --output TOBIAS/PlotAggregate/DOF4.7_AT4G38000_footprint_comparison_bound.pdf --share_y both --plot_boundaries

# TOBIAS 0.16.1 PlotAggregate (run started 2024-01-30 14:20:20.967314)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotAggregate --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bound.bed TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed --signals TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --output TOBIAS/PlotAggregate/DOF4.7_AT4G38000_footprint_comparison_bound.pdf --share_y both --plot_boundaries

# ----- Input parameters -----
# TFBS:	['TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bound.bed', 'TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed']
# signals:	['TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw', 'TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw']
# regions:	[]
# whitelist:	[]
# blacklist:	[]
# output:	TOBIAS/PlotAggregate/DOF4.7_AT4G38000_footprint_co

In [117]:
def compare_bound_between_two_conditions(tf_name):
    # Define the base path for BED files and signal files
    bed_file_dm = f"TOBIAS/BINDetect_output_v3/{tf_name}/beds/{tf_name}_DM_bound.bed"
    bed_file_col = f"TOBIAS/BINDetect_output_v3/{tf_name}/beds/{tf_name}_COL_bound.bed"
    signal_dm = "TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw"
    signal_col = "TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw"

    # Define the command for PlotAggregate
    cmd = [
        "TOBIAS", "PlotAggregate", 
        "--TFBS", bed_file_dm, bed_file_col,
        "--signals", signal_dm, signal_col,
        "--output", f"TOBIAS/PlotAggregate/{tf_name}_footprint_comparison_bound.pdf",
        "--share_y", "both", 
        "--plot_boundaries", 
        "--signal-on-x"
    ]

    # Execute the command
    try:
        subprocess.run(cmd, check=True)
        return f'TOBIAS PlotAggregate for {tf_name} comparing bound sites between conditions Success'
    except subprocess.CalledProcessError as e:
        return f"An error occurred: {e}"
compare_bound_between_two_conditions("WRKY18_AT4G31800")#
compare_bound_between_two_conditions("NAC046_AT3G04060")

# TOBIAS 0.16.1 PlotAggregate (run started 2024-03-20 15:28:16.070152)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotAggregate --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed --signals TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --output TOBIAS/PlotAggregate/WRKY18_AT4G31800_footprint_comparison_bound.pdf --share_y both --plot_boundaries --signal-on-x

# ----- Input parameters -----
# TFBS:	['TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_bound.bed', 'TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed']
# signals:	['TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw', 'TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw']
# regions:	[]
# whitelist:	[]
# blacklist:	[]
# output:	TOBIAS/PlotAggregate/WRKY18_AT4G3180

'TOBIAS PlotAggregate for NAC046_AT3G04060 comparing bound sites between conditions Success'

In [82]:
import subprocess

def compare_bound_unbound_in_one_condition(tf_name, condition):
    # Validate the condition
    if condition not in ['DM', 'COL']:
        return "Invalid condition. Choose 'DM' or 'COL'."

    # Define the base path for BED files
    bed_base_path = f"TOBIAS/BINDetect_output_v3/{tf_name}/beds/{tf_name}"

    # Define the BED files for all, bound, and unbound
    bed_files = [
        f"{bed_base_path}_all.bed",
        f"{bed_base_path}_{condition}_bound.bed",
        f"{bed_base_path}_{condition}_unbound.bed"
    ]

    # Define signal files based on the condition, adjusting the paths
    if condition == 'DM':
        signal_base_path = "TOBIAS/ATACorrect_DM"
        signal_files = [
            f"{signal_base_path}/DM1_DM2d_DM6_DM7_uncorrected.bw",
            f"{signal_base_path}/DM1_DM2d_DM6_DM7_expected.bw",
            f"{signal_base_path}/DM1_DM2d_DM6_DM7_corrected.bw"
        ]
    else: # condition == 'COL'
        signal_base_path = "TOBIAS/ATACorrect_C12_C14"
        signal_files = [
            f"{signal_base_path}/C12_C14_merged_uncorrected.bw",
            f"{signal_base_path}/C12_C14_merged_expected.bw",
            f"{signal_base_path}/C12_C14_merged_corrected.bw"
        ]

    # Define the command for PlotAggregate
    cmd = [
        "TOBIAS", "PlotAggregate", 
        "--TFBS"] + bed_files + [
        "--signals"] + signal_files + [
        "--output", f"TOBIAS/PlotAggregate/{tf_name}_footprint_comparison_{condition}.pdf",
        "--share_y", "both", 
        "--plot_boundaries", 
        "--signal-on-x"
    ]

    # Execute the command
    try:
        subprocess.run(cmd, check=True)
        return f'TOBIAS PlotAggregate for {tf_name} in {condition} condition Success'
    except subprocess.CalledProcessError as e:
        return f"An error occurred: {e}"

# Example usage
compare_result = compare_bound_unbound_in_one_condition("NLP4_AT1G20640", "COL")
# print(compare_result)


# TOBIAS 0.16.1 PlotAggregate (run started 2024-03-14 13:22:25.550810)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotAggregate --TFBS TOBIAS/BINDetect_output_v3/NLP4_AT1G20640/beds/NLP4_AT1G20640_all.bed TOBIAS/BINDetect_output_v3/NLP4_AT1G20640/beds/NLP4_AT1G20640_COL_bound.bed TOBIAS/BINDetect_output_v3/NLP4_AT1G20640/beds/NLP4_AT1G20640_COL_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_uncorrected.bw TOBIAS/ATACorrect_C12_C14/C12_C14_merged_expected.bw TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --output TOBIAS/PlotAggregate/NLP4_AT1G20640_footprint_comparison_COL.pdf --share_y both --plot_boundaries --signal-on-x

# ----- Input parameters -----
# TFBS:	['TOBIAS/BINDetect_output_v3/NLP4_AT1G20640/beds/NLP4_AT1G20640_all.bed', 'TOBIAS/BINDetect_output_v3/NLP4_AT1G20640/beds/NLP4_AT1G20640_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/NLP4_AT1G20640/beds/NLP4_AT1G20640_COL_unbound.bed']
# signals:	['TOBIAS/ATACorre

In [116]:
! TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bound.bed TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_unbound.bed --signals TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --output TOBIAS/PlotAggregate/DOF4.7_AT4G38000_heatmap.png --sort_by -2

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-20 15:27:33.665595)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bound.bed TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_unbound.bed --signals TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw --output TOBIAS/PlotAggregate/DOF4.7_AT4G38000_heatmap.png --sort_by -2

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bound.bed', 'TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_unbound.bed'], ['TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed', 'TOBIAS/BINDete

In [118]:
def run_plot_heatmap(tf_name, width, height):
    print("Running TOBIAS PlotHeatmap")

    # Define the output directory and ensure it exists
    output_dir = "TOBIAS/PlotHeatmap"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define the base path for BED files
    bed_base_path = f"TOBIAS/BINDetect_output_v3/{tf_name}/beds/{tf_name}"

    # Define the command for PlotHeatmap
    cmd = [
        "python", "plot_heatmap2.py",
        "--TFBS", f"{bed_base_path}_COL_bound.bed", f"{bed_base_path}_COL_unbound.bed",
        "--TFBS", f"{bed_base_path}_DM_bound.bed", f"{bed_base_path}_DM_unbound.bed",
        "--signals", "TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw", "TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw",
        "--output", os.path.join(output_dir, f"{tf_name}_heatmap.pdf"),
        "--sort_by", "-2",
        "--width", str(width),
        "--height", str(height),
    ]

    print(f"Running command: {' '.join(cmd)}")

    # Execute the command
    try:
        subprocess.run(cmd, check=True)
        print(f"TOBIAS PlotHeatmap for {tf_name} Success")
        return f'TOBIAS PlotHeatmap for {tf_name} Success'
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        return f"An error occurred: {e}"

# Example usage
tf_name = "WRKY18_AT4G31800"
width = 3
height = 3
plot_heatmap_result = run_plot_heatmap(tf_name, width, height)

Running TOBIAS PlotHeatmap
Running command: python plot_heatmap2.py --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/WRKY18_AT4G31800_heatmap.pdf --sort_by -2 --width 3 --height 3
loaded libraries
Executing plot_heatmap2.py script
Calling run_heatmap function
Entering run_heatmap function
# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-20 15:28:29.201390)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT

Traceback (most recent call last):
  File "/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/plot_heatmap2.py", line 120, in run_heatmap
    heatmap_info[col][row]["regions"].sort(key=lambda region: float(region[args.sort_by]), reverse=True)
  File "/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/plot_heatmap2.py", line 120, in <lambda>
    heatmap_info[col][row]["regions"].sort(key=lambda region: float(region[args.sort_by]), reverse=True)
TypeError: list indices must be integers or slices, not str

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/plot_heatmap2.py", line 293, in <module>
    run_heatmap(args)
  File "/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/plot_heatmap2.py", line 122, in run_heatmap
    heatmap_info[col][row]["regions"].sort(key=lambda region: region[args.sort_by], reverse=True)
  File "/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/plot_heatmap2.py", line 122, in <lambda>
    heatmap_info[col][ro

In [115]:
! python plot_heatmap2.py --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/WRKY18_AT4G31800_heatmap.pdf --sort_by -2 --width 3 --height 3

loaded libraries


### this!

In [123]:
import subprocess
import os

def run_plot_heatmap(tf_name):
    # Define the output directory and ensure it exists
    output_dir = "TOBIAS/PlotHeatmap"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define the base path for BED files
    bed_base_path = f"TOBIAS/BINDetect_output_v3/{tf_name}/beds/{tf_name}"

    # # Define the BED files for bound and unbound in DM and COL conditions
    # bed_files = [
    #     f"{bed_base_path}_COL_bound.bed", f"{bed_base_path}_COL_unbound.bed",
    #       f"{bed_base_path}_DM_bound.bed", f"{bed_base_path}_DM_unbound.bed"
    # ]

    # Define the command for PlotHeatmap
    cmd = [
        "TOBIAS", "PlotHeatmap", 
        "--TFBS"] + [ f"{bed_base_path}_COL_bound.bed", f"{bed_base_path}_COL_unbound.bed",] + [
        "--TFBS"] + [ f"{bed_base_path}_DM_bound.bed", f"{bed_base_path}_DM_unbound.bed",] + [
        "--signals", "TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw", "TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw", 
        "--output", os.path.join(output_dir, f"{tf_name}_heatmap.pdf"),
        "--sort_by", "-2", '--title', tf_name
    ]

    # Execute the command
    try:
        subprocess.run(cmd, check=True)
        return f'TOBIAS PlotHeatmap for {tf_name} Success'
        print(f"PlotHeatmap for {tf_name} Success")
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        return f"An error occurred: {e}"

# Example usage
tf_name = "WRKY18_AT4G31800"
plot_heatmap_result = run_plot_heatmap(tf_name)



# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-20 18:06:31.486696)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/WRKY18_AT4G31800_heatmap.pdf --sort_by -2 --title WRKY18_AT4G31800

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_bou

In [125]:
! TOBIAS PlotHeatmap

__________________________________________________________________________________________

                                   TOBIAS ~ PlotHeatmap                                   
__________________________________________________________________________________________

PlotHeatmap plots a heatmap of signals from bigwig(s) (each row is one site) as well as
the aggregate signal across all sites.

------------------------------------------------------------------------------------------

Input / output arguments:
  --TFBS [ ...]           TFBS sites per column (*required)
  --signals [ ...]        Signals in bigwig format (*required)
  --output                Output filename (default: TOBIAS_heatmap.pdf)

Plot arguments:
  --plot-boundaries       Plot TFBS boundaries
  --share-colorbar        Share colorbar across all bigwigs (default: estimate colorbar
                          per bigwig)
  --flank 
  --title 
  --TFBS-labels [ ...]    Labels of TFBS (default: basename of --TFBS)
 

In [124]:
import matplotlib.pyplot as plt

# List all available colormaps
colormaps = plt.colormaps()
print(colormaps)


['magma', 'inferno', 'plasma', 'viridis', 'cividis', 'twilight', 'twilight_shifted', 'turbo', 'Blues', 'BrBG', 'BuGn', 'BuPu', 'CMRmap', 'GnBu', 'Greens', 'Greys', 'OrRd', 'Oranges', 'PRGn', 'PiYG', 'PuBu', 'PuBuGn', 'PuOr', 'PuRd', 'Purples', 'RdBu', 'RdGy', 'RdPu', 'RdYlBu', 'RdYlGn', 'Reds', 'Spectral', 'Wistia', 'YlGn', 'YlGnBu', 'YlOrBr', 'YlOrRd', 'afmhot', 'autumn', 'binary', 'bone', 'brg', 'bwr', 'cool', 'coolwarm', 'copper', 'cubehelix', 'flag', 'gist_earth', 'gist_gray', 'gist_heat', 'gist_ncar', 'gist_rainbow', 'gist_stern', 'gist_yarg', 'gnuplot', 'gnuplot2', 'gray', 'hot', 'hsv', 'jet', 'nipy_spectral', 'ocean', 'pink', 'prism', 'rainbow', 'seismic', 'spring', 'summer', 'terrain', 'winter', 'Accent', 'Dark2', 'Paired', 'Pastel1', 'Pastel2', 'Set1', 'Set2', 'Set3', 'tab10', 'tab20', 'tab20b', 'tab20c', 'magma_r', 'inferno_r', 'plasma_r', 'viridis_r', 'cividis_r', 'twilight_r', 'twilight_shifted_r', 'turbo_r', 'Blues_r', 'BrBG_r', 'BuGn_r', 'BuPu_r', 'CMRmap_r', 'GnBu_r', 'G

In [106]:
run_plot_heatmap('DOF4.7_AT4G38000')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-14 23:43:27.668495)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bound.bed TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/DOF4.7_AT4G38000_heatmap.pdf --sort_by -2 --title DOF4.7_AT4G38000

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/DOF4.7_AT4G38000/beds/DOF4.7_AT4G38000_DM_bou

'TOBIAS PlotHeatmap for DOF4.7_AT4G38000 Success'

In [107]:
run_plot_heatmap('WRKY46_AT2G46400')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-14 23:43:46.524501)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_COL_bound.bed TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_DM_bound.bed TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/WRKY46_AT2G46400_heatmap.pdf --sort_by -2 --title WRKY46_AT2G46400

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/WRKY46_AT2G46400/beds/WRKY46_AT2G46400_DM_bou

'TOBIAS PlotHeatmap for WRKY46_AT2G46400 Success'

In [108]:
run_plot_heatmap('NAC058_AT3G18400')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-14 23:43:52.749742)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_COL_bound.bed TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_DM_bound.bed TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/NAC058_AT3G18400_heatmap.pdf --sort_by -2 --title NAC058_AT3G18400

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/NAC058_AT3G18400/beds/NAC058_AT3G18400_DM_bou

'TOBIAS PlotHeatmap for NAC058_AT3G18400 Success'

In [109]:
run_plot_heatmap('AT5G01380_AT5G01380')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-14 23:43:58.234722)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_COL_bound.bed TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_DM_bound.bed TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/AT5G01380_AT5G01380_heatmap.pdf --sort_by -2 --title AT5G01380_AT5G01380

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/AT5G01380_AT5G01380/beds/AT5G01380_AT5G01380_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/AT5

'TOBIAS PlotHeatmap for AT5G01380_AT5G01380 Success'

In [110]:
run_plot_heatmap('CDF5_AT1G69570')

# TOBIAS 0.16.1 PlotHeatmap (run started 2024-03-14 23:44:03.018867)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotHeatmap --TFBS TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_COL_bound.bed TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_DM_bound.bed TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/CDF5_AT1G69570_heatmap.pdf --sort_by -2 --title CDF5_AT1G69570

# ----- Input parameters -----
# TFBS:	[['TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_COL_bound.bed', 'TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_COL_unbound.bed'], ['TOBIAS/BINDetect_output_v3/CDF5_AT1G69570/beds/CDF5_AT1G69570_DM_bound.bed', 'TOBIAS/BINDetect_outpu

'TOBIAS PlotHeatmap for CDF5_AT1G69570 Success'

In [87]:
! TOBIAS PlotHeatmap 

__________________________________________________________________________________________

                                   TOBIAS ~ PlotHeatmap                                   
__________________________________________________________________________________________

PlotHeatmap plots a heatmap of signals from bigwig(s) (each row is one site) as well as
the aggregate signal across all sites.

------------------------------------------------------------------------------------------

Input / output arguments:
  --TFBS [ ...]           TFBS sites per column (*required)
  --signals [ ...]        Signals in bigwig format (*required)
  --output                Output filename (default: TOBIAS_heatmap.pdf)

Plot arguments:
  --plot-boundaries       Plot TFBS boundaries
  --share-colorbar        Share colorbar across all bigwigs (default: estimate colorbar
                          per bigwig)
  --flank 
  --title 
  --TFBS-labels [ ...]    Labels of TFBS (default: basename of --TFBS)
 

In [99]:
def run_plot_heatmap2(tf_name, width, height):
    # Define the output directory and ensure it exists
    output_dir = "TOBIAS/PlotHeatmap"
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Define the base path for BED files
    bed_base_path = f"TOBIAS/BINDetect_output_v3/{tf_name}/beds/{tf_name}"

    # Define the command for PlotHeatmap
    cmd = [
        "python", "plot_heatmap2.py",
        "--TFBS", f"{bed_base_path}_COL_bound.bed", f"{bed_base_path}_COL_unbound.bed",
        "--TFBS", f"{bed_base_path}_DM_bound.bed", f"{bed_base_path}_DM_unbound.bed",
        "--signals", "TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw", "TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw",
        "--output", os.path.join(output_dir, f"{tf_name}_heatmap.pdf"),
        "--sort_by", "-2",
        "--width", str(width),
        "--height", str(height),
    ]

    # Execute the command
    try:
        subprocess.run(cmd, check=True)
        return f'TOBIAS PlotHeatmap for {tf_name} Success'
    except subprocess.CalledProcessError as e:
        print(f"An error occurred: {e}")
        return f"An error occurred: {e}"
    
tf_name = "WRKY18_AT4G31800"
width = 3
height = 3
plot_heatmap_result = run_plot_heatmap2(tf_name, width, height)
plot_heatmap_result

'TOBIAS PlotHeatmap for WRKY18_AT4G31800 Success'

In [100]:
! python plot_heatmap2.py --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_COL_unbound.bed --TFBS TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_bound.bed TOBIAS/BINDetect_output_v3/WRKY18_AT4G31800/beds/WRKY18_AT4G31800_DM_unbound.bed --signals TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --output TOBIAS/PlotHeatmap/WRKY18_AT4G31800_heatmap.pdf --sort_by -2 --width 3 --height 3

### CreateNetwork

In [3]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/*/beds/*_all.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_all


# TOBIAS 0.16.1 CreateNetwork (run started 2024-01-24 20:06:44.040306)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF1_AT1G49720/beds/ABF1_AT1G49720_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF2_AT1G45249/beds/ABF2_AT1G45249_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF3_AT4G34000/beds/ABF3_AT4G34000_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI3_AT3G24650/beds/ABI3_AT3G24650_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI4_AT2G40220/beds/ABI4_AT2G40220_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI5_AT2G36270/beds/ABI5_AT2G36270_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABO3_AT1G66600/beds/ABO3_AT1G66600_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_o

In [2]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/*/beds/*DM_bound.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_DM


# TOBIAS 0.16.1 CreateNetwork (run started 2024-01-24 20:16:23.248964)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF1_AT1G49720/beds/ABF1_AT1G49720_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF2_AT1G45249/beds/ABF2_AT1G45249_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABF3_AT4G34000/beds/ABF3_AT4G34000_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI3_AT3G24650/beds/ABI3_AT3G24650_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI4_AT2G40220/beds/ABI4_AT2G40220_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABI5_AT2G36270/beds/ABI5_AT2G36270_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/ABO3_AT1G66600/beds/ABO3_AT1G66600_DM_bound.bed /media/HDD2/donghui/bu

In [ ]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_v3/*/beds/*COL_bound.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_COL


#### Using all 600 TFs in the BINDetect is too much, use only DEG  l2f = 2

In [30]:
import pyreadr
import pandas as pd
import re

# Load the RDS file
result = pyreadr.read_r('../3DM_shoot/Rds_Rdata/Res_3DM.2023-04-21 14:05:06rds')

# Extract the DataFrame
df = result[None]

# Filter rows where 'contrast' column contains 'DM1/DM2d'
filtered_df = df[df['contrast'].str.contains('DM1/DM2d', na=False)]

# Select the 'AGI' and 'log2FoldChange' columns
df_DM1_DM2d = filtered_df[['AGI', 'log2FoldChange']]


In [31]:
#filter log2FoldChange abs > 2 
df_DM1_DM2d = df_DM1_DM2d[df_DM1_DM2d['log2FoldChange'].abs() > 2]
df_DM1_DM2d

,AGI,log2FoldChange
26369,AT1G30814,5.347096
26371,AT1G71695,-3.323255
26379,AT1G65630,-3.848987
26380,AT1G71390,3.127214
26384,AT1G73360,-3.777769
...,...,...
52724,ATCG00990,-2.809187
52725,ATCG01200,-4.638700
52727,ATCG01290,-2.920030
52728,ATCG00410,-2.612143


In [36]:
# Assuming df_DM1_DM2d is your DataFrame
agi_list = df_DM1_DM2d['AGI'].tolist()

# Join the elements with a space
agi_str = ' '.join(agi_list)

# Write to a text file
with open('./TOBIAS/meme/DEG_TFs.txt', 'w') as file:
    file.write(agi_str)

### the activators and repressors in network

In [132]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('/media/HDD2/donghui/bulk_ATAC_DM1_DM2d/diffbind_results_2DM/nodes_diffTF_AME_WGCNA_modules_L2fold_abs.csv')

# Filter the DataFrame for 'activator' or 'repressor' in the 'TF_class' column
filtered_df = df[df['TF_class'].isin(['activator', 'repressor'])]

# Convert the 'ID' column to a list
id_list = filtered_df['ID'].tolist()

# Convert the list to a string with space-separated IDs
id_string = ' '.join(id_list)

# Write the string to a text file and ensure it ends with a newline
with open('./TOBIAS/meme/activators_or_repressors.txt', 'w') as file:
    file.write(id_string + '\n')



In [37]:
# ! echo 'AT3G56400' > ./TOBIAS/meme/DEG_TFs.txt
! TOBIAS FormatMotifs --input ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme --output ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar --filter ./TOBIAS/meme/DEG_TFs.txt

# TOBIAS 0.16.1 FormatMotifs (run started 2024-01-26 15:25:12.879348)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS FormatMotifs --input ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme --output ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar --filter ./TOBIAS/meme/DEG_TFs.txt

# ----- Input parameters -----
# input:	['./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme']
# output:	./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar
# format:	jaspar
# task:	join
# filter:	./TOBIAS/meme/DEG_TFs.txt
# verbosity:	3


# ----- Output files -----
# ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar


2024-01-26 15:25:12 (1037664) [INFO]	Reading input files...
2024-01-26 15:25:12 (1037664) [INFO]	Read 619 motifs

2024-01-26 15:25:12 (1037664) [INFO]	Reading entries in ./TOBIAS/meme/DEG_TFs.txt
2024-01-26 15:25:12 (1037664) [INFO]	Read 8805 unique filter values
2024-01-26 15:25

In [134]:
! TOBIAS FormatMotifs --input ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme --output ./TOBIAS/meme/activators_or_repressors.jaspar --filter ./TOBIAS/meme/activators_or_repressors.txt

# TOBIAS 0.16.1 FormatMotifs (run started 2024-04-11 22:04:29.144537)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS FormatMotifs --input ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme --output ./TOBIAS/meme/activators_or_repressors.jaspar --filter ./TOBIAS/meme/activators_or_repressors.txt

# ----- Input parameters -----
# input:	['./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB.meme']
# output:	./TOBIAS/meme/activators_or_repressors.jaspar
# format:	jaspar
# task:	join
# filter:	./TOBIAS/meme/activators_or_repressors.txt
# verbosity:	3


# ----- Output files -----
# ./TOBIAS/meme/activators_or_repressors.jaspar


2024-04-11 22:04:29 (679471) [INFO]	Reading input files...
2024-04-11 22:04:29 (679471) [INFO]	Read 619 motifs

2024-04-11 22:04:29 (679471) [INFO]	Reading entries in ./TOBIAS/meme/activators_or_repressors.txt
2024-04-11 22:04:29 (679471) [INFO]	Read 63 unique filter values
2024-04-11 22:04:29 (679471) [INFO]	Matchin

In [ ]:
TOBIAS BINDetect --motifs ./TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar --signals TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --genome /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --peaks TOBIAS/UROPA/merged_peaks_annotated.bed --peak_header TOBIAS/UROPA/merged_peaks_annotated_header.txt --outdir TOBIAS/BINDetect_output_DEG --cond_names COL DM --cores 99

### CreateNetwork

In [ ]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/*/beds/*_all.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_DEG_all


# TOBIAS 0.16.1 CreateNetwork (run started 2024-01-26 15:44:26.778598)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABI3_AT3G24650/beds/ABI3_AT3G24650_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABO3_AT1G66600/beds/ABO3_AT1G66600_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABR1_AT5G64750/beds/ABR1_AT5G64750_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AFO_AT2G45190/beds/AFO_AT2G45190_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AG_AT4G18960/beds/AG_AT4G18960_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AGL13_AT3G61120/beds/AGL13_AT3G61120_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AGL6_AT2G45650/beds/AGL6_AT2G45650_all.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetec

In [1]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/*/beds/*DM_bound.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_DEG_DM


# TOBIAS 0.16.1 CreateNetwork (run started 2024-01-26 15:36:29.735174)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABI3_AT3G24650/beds/ABI3_AT3G24650_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABO3_AT1G66600/beds/ABO3_AT1G66600_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/ABR1_AT5G64750/beds/ABR1_AT5G64750_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AFO_AT2G45190/beds/AFO_AT2G45190_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AG_AT4G18960/beds/AG_AT4G18960_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AGL13_AT3G61120/beds/AGL13_AT3G61120_DM_bound.bed /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/AGL6_AT2G45650/beds/AGL6_AT2G45650_DM_bound.bed /media/HDD2/donghui

In [ ]:
! TOBIAS CreateNetwork --TFBS /media/HDD2/donghui/bulk_ATAC_DM1_DM2d/TOBIAS/BINDetect_output_DEG/*/beds/*COL_bound.bed --origin ./TOBIAS/meme/motif2gene_mapping.txt --outdir ./TOBIAS/CreateNetwork_DEG_COL


#### TFBS

In [53]:
! TOBIAS TFBScan --motifs TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar --fasta /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --cores 80 --outdir TOBIAS/TFBScan_DEG

# TOBIAS 0.16.1 TFBScan (run started 2024-01-30 16:51:19.789346)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS TFBScan --motifs TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar --fasta /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --cores 80 --outdir TOBIAS/TFBScan_DEG

# ----- Input parameters -----
# motifs:	TOBIAS/meme/updated_Ath_TF_binding_motifs_plantTFDB_only_DEG.jaspar
# fasta:	/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa
# regions:	None
# outdir:	TOBIAS/TFBScan_DEG
# outfile:	None
# naming:	name_id
# gc:	None
# pvalue:	0.0001
# keep_overlaps:	False
# add_region_columns:	False
# verbosity:	3
# split:	100
# cores:	80


2024-01-30 16:51:19 (1639287) [INFO]	Handling input files
2024-01-30 16:51:19 (1639287) [INFO]	Reading sequences from fasta
2024-01-30 16:51:19 (1639287) [STATS]	- Fo

In [135]:
! TOBIAS TFBScan --motifs ./TOBIAS/meme/activators_or_repressors.jaspar --fasta /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --cores 80 --outdir TOBIAS/TFBScan_activators_or_repressors

# TOBIAS 0.16.1 TFBScan (run started 2024-04-11 22:05:20.934624)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS TFBScan --motifs ./TOBIAS/meme/activators_or_repressors.jaspar --fasta /media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa --cores 80 --outdir TOBIAS/TFBScan_activators_or_repressors

# ----- Input parameters -----
# motifs:	./TOBIAS/meme/activators_or_repressors.jaspar
# fasta:	/media/HDD2/donghui/diffTF_ath/diffTF/example/stable/input/referenceGenome/Arabidopsis_thaliana.TAIR10.modified.fa
# regions:	None
# outdir:	TOBIAS/TFBScan_activators_or_repressors
# outfile:	None
# naming:	name_id
# gc:	None
# pvalue:	0.0001
# keep_overlaps:	False
# add_region_columns:	False
# verbosity:	3
# split:	100
# cores:	80


2024-04-11 22:05:20 (679646) [INFO]	Handling input files
2024-04-11 22:05:20 (679646) [INFO]	Reading sequences from fasta
2024-04-11 22:05:20 (679646) [STATS]	- Found 7

In [56]:
! cat TOBIAS/TFBScan_DEG/*.bed | sort -k1,1 -k2,2n | uniq > TOBIAS/TFBScan_DEG/sorted_nodup_merged.bed

In [136]:
! cat TOBIAS/TFBScan_activators_or_repressors/*.bed | sort -k1,1 -k2,2n | uniq > TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_merged.bed

In [59]:
! cat TOBIAS/TFBScan_DEG/WRKY*.bed | sort -k1,1 -k2,2n | uniq > TOBIAS/TFBScan_DEG/WRKY_sorted_nodup_merged.bed

In [67]:
! cat TOBIAS/TFBScan_DEG/DOF*.bed | sort -k1,1 -k2,2n | uniq > TOBIAS/TFBScan_DEG/DOF_sorted_nodup_merged.bed

### ! TOBIAS PlotTracks

In [48]:
! echo -e "chr1\t12166281\t12169492" > ./TOBIAS/regions/ADR1.bed

In [45]:
$ TOBIAS PlotTracks --bigwigs test_data/*cell_corrected.bw --bigwigs test_data/*cell_footprints.bw --regions test_data/plot_regions.bed --sites test_data/binding_sites.bed --highlight test_data/binding_sites.bed --gtf test_data/genes.gtf --colors red darkblue red darkblue

SyntaxError: invalid syntax (578903902.py, line 1)

In [127]:
! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/*bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/sorted_nodup_merged.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks/ADR1_all

# TOBIAS 0.16.1 PlotTracks (run started 2024-04-06 22:39:14.823527)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/C12_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw TOBIAS/Footprints/DM1_DM2d_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/sorted_nodup_merged.bed --colors red darkblue red darkblue --outdir TOBIAS/PlotTracks/ADR1_all

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/C12_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw', 'TOBIAS/Footprints/DM1_DM2d_foorprint.b

### pADR1 all 2kb

In [139]:
# ! echo -e "chr1\t12166281\t12169492" > ./TOBIAS/regions/ADR1.bed
! echo -e "chr1\t12166767\t12168867" > ./TOBIAS/regions/ADR1.bed

! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_merged.bed --colors darkblue red darkblue red --outdir TOBIAS/PlotTracks/ADR1_all_2kb_activators_or_repressors

# TOBIAS 0.16.1 PlotTracks (run started 2024-04-11 22:57:23.754581)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_merged.bed --colors darkblue red darkblue red --outdir TOBIAS/PlotTracks/ADR1_all_2kb_activators_or_repressors

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/ADR1.bed
# sites:	TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_merged

In [141]:
# ! echo -e "chr1\t12166281\t12169492" > ./TOBIAS/regions/ADR1.bed
! echo -e "chr1\t12166767\t12168867" > ./TOBIAS/regions/ADR1.bed

! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_WRKYs_NACs_cleaned.bed --colors darkblue red darkblue red --outdir TOBIAS/PlotTracks/ADR1_all_2kb_activators_or_repressors

# TOBIAS 0.16.1 PlotTracks (run started 2024-04-12 09:19:16.769143)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_WRKYs_NACs_cleaned.bed --colors darkblue red darkblue red --outdir TOBIAS/PlotTracks/ADR1_all_2kb_activators_or_repressors

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/ADR1.bed
# sites:	TOBIAS/TFBScan_DEG/TFBScan_activators_or_repre

In [146]:
# ! echo -e "chr1\t12166281\t12169492" > ./TOBIAS/regions/ADR1.bed
! echo -e "chr1\t12166561\t12169040" > ./TOBIAS/regions/ADR1.bed

! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_WRKYs_NACs_cleaned.bed --colors darkblue red darkblue red --outdir TOBIAS/PlotTracks/ADR1_all_2kb_activators_or_repressors_as_pADR1

# TOBIAS 0.16.1 PlotTracks (run started 2024-04-12 13:09:56.183975)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/ADR1.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_WRKYs_NACs_cleaned.bed --colors darkblue red darkblue red --outdir TOBIAS/PlotTracks/ADR1_all_2kb_activators_or_repressors_as_pADR1

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/ADR1.bed
# sites:	TOBIAS/TFBScan_DEG/TFBScan_activators

In [143]:
! echo -e "chr1\t22387257\t22389457" > ./TOBIAS/regions/NIK3.bed #Chr1:22383452..22387457

! TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/NIK3.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_WRKYs_NACs_cleaned.bed --colors darkblue red darkblue red --outdir TOBIAS/PlotTracks/NIK3

# TOBIAS 0.16.1 PlotTracks (run started 2024-04-12 09:40:50.384894)
# Working directory: /media/HDD2/donghui/bulk_ATAC_DM1_DM2d
# Command line call: TOBIAS PlotTracks --bigwigs TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw --bigwigs TOBIAS/Footprints/C12_C14_footprint.bw TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw --regions TOBIAS/regions/NIK3.bed --gtf TOBIAS/gff3/Arabidopsis_thaliana.TAIR10.58_modified.gtf --sites TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_WRKYs_NACs_cleaned.bed --colors darkblue red darkblue red --outdir TOBIAS/PlotTracks/NIK3

# ----- Input parameters -----
# bigwigs:	[['TOBIAS/ATACorrect_C12_C14/C12_C14_merged_corrected.bw', 'TOBIAS/ATACorrect_DM/DM1_DM2d_DM6_DM7_corrected.bw'], ['TOBIAS/Footprints/C12_C14_footprint.bw', 'TOBIAS/Footprints/DM1_DM2d_DM6_DM7_foorprint.bw']]
# regions:	TOBIAS/regions/NIK3.bed
# sites:	TOBIAS/TFBScan_DEG/TFBScan_activators_or_repressors_WRKYs_NACs_cleaned.bed
# hi